In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
from tqdm import tqdm
import csv

In [2]:
dates = ['2017-11-26', '2017-11-27', '2017-11-28', '2017-11-29', '2017-11-30', '2017-12-01' , '2017-12-02', '2017-12-03'] # '2017-11-25', 

In [3]:
NN_tables = []
for part in range(1, 5):
    with open('../Taobao_v3/dict_NN_table/NN_table_'+ str(part) +'.pkl', 'rb') as file:
        NN_tables.append(pickle.load(file))

In [4]:
def NN_ranking(r):
    for part in range(4):
        if str(r.target_iid) in NN_tables[part]:
            lt = NN_tables[part][str(r.target_iid)]
            ids, score = zip(*lt)
            try:
                rank = ids.index(str(r.next_iid)) + 1
            except:
                rank = np.nan
            break
    else:
        rank = np.nan
    return rank

In [5]:
lt_df = []
for date in tqdm(dates):
    with open('../DIEN_T/testing_table/' + date + '_knn.pkl', 'rb') as file:
        df = pickle.load(file)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['NN_rank'] = df.apply(NN_ranking, axis=1)
    
    dff = pd.read_csv('./din_ali/' + 'results_' + date[5:] + '.csv')
    dff.rename(columns = {'rank':'DIN_rank'}, inplace=True)
    dff['uid'] = dff.apply(lambda df: eval(df['uid'])[0], axis=1)
    dff.sort_values(by=['uid'], inplace=True, ignore_index=True)
    dff.drop(columns=['hist_iid', 'last_iid', 'target_iid'], inplace=True)
    try:
        pd.testing.assert_series_equal(df['uid'], dff['uid'])
        df['DIN_rank'] = dff['DIN_rank']
    except Exception as e:
        print(date)
        print(e)
        df = pd.merge(df, dff, on=['uid'], how='left')
        df.fillna(dff['DIN_rank'].mean(), inplace=True)

    dff = pd.read_csv('./dien_ali/' + 'ali_dien_rank_' + date[5:] + '.csv')
    dff.rename(columns = {'dien_rank':'DIEN_rank'}, inplace=True)
#     dff['uid'] = dff.apply(lambda df: eval(df['uid'])[0], axis=1)
    dff.sort_values(by=['uid'], inplace=True, ignore_index=True)
    dff.drop(columns=['hist_iid', 'last_id', 'target_iid'], inplace=True)
    pd.testing.assert_series_equal(df['uid'], dff['uid'])
    df['DIEN_rank'] = dff['DIEN_rank']

    lt_df.append(df)

 38%|███▊      | 3/8 [00:25<00:45,  9.10s/it]

2017-11-28
Series are different

Series length are different
[left]:  13030, RangeIndex(start=0, stop=13030, step=1)
[right]: 13029, RangeIndex(start=0, stop=13029, step=1)


 50%|█████     | 4/8 [00:32<00:34,  8.66s/it]

2017-11-29
Series are different

Series length are different
[left]:  12875, RangeIndex(start=0, stop=12875, step=1)
[right]: 12874, RangeIndex(start=0, stop=12874, step=1)


 75%|███████▌  | 6/8 [00:48<00:16,  8.23s/it]

2017-12-01
Series are different

Series length are different
[left]:  12994, RangeIndex(start=0, stop=12994, step=1)
[right]: 12993, RangeIndex(start=0, stop=12993, step=1)


 88%|████████▊ | 7/8 [00:56<00:08,  8.32s/it]

2017-12-02
Series are different

Series length are different
[left]:  16059, RangeIndex(start=0, stop=16059, step=1)
[right]: 16058, RangeIndex(start=0, stop=16058, step=1)


100%|██████████| 8/8 [01:05<00:00,  8.16s/it]

2017-12-03
Series are different

Series length are different
[left]:  15364, RangeIndex(start=0, stop=15364, step=1)
[right]: 15362, RangeIndex(start=0, stop=15362, step=1)


In [7]:
with open('./lt_df_Ali_8days.pkl', 'wb') as file:
    pickle.dump(lt_df, file)

In [8]:
lt_df[0]

,uid,hist_iid,hist_cid,target_iid,target_cid,next_iid,next_cid,knn_rank,NN_rank,DIN_rank,DIEN_rank
0,7,"[980099, 3427154, 2689961, 2292610, 4034225, 5...","[2355072, 4801426, 4756105, 4756105, 3607361, ...",2063176,149192,1793668,2520377,53.0,40,95,45
1,96,"[3005381, 4160584]","[1464116, 3607361]",4160584,3607361,13283,3607361,5.0,7,15,94
2,152,"[842950, 3048082, 3682882]","[4145813, 4181361, 4181361]",3682882,4181361,3739757,4181361,6.0,6,45,18
3,239,"[709710, 4826455, 709710, 1992230, 1842641, 45...","[1787510, 1787510, 1787510, 2465336, 2976589, ...",5030834,1787510,24834,1787510,28.0,16,89,11
4,339,"[3437026, 3523692, 1850821, 1687195, 2487310, ...","[3738615, 2558244, 4756105, 4022701, 3738615, ...",222342,4244487,1692072,4244487,4.0,1,50,61
...,...,...,...,...,...,...,...,...,...,...,...
14738,1017455,"[3878908, 2029728, 4877816, 4736367, 1387695]","[982926, 149192, 4801426, 5042232, 4756105]",1387695,4756105,1584827,4756105,2.0,24,10,20
14739,1017559,"[969064, 7521, 1367959, 812391, 4257874]","[903809, 903809, 903809, 903809, 903809]",4257874,903809,2736720,903809,34.0,80,40,39
14740,1017591,"[777173, 2503250, 4551433]","[2885642, 2885642, 2885642]",4551433,2885642,2157244,2885642,38.0,91,3,51
14741,1017830,"[2382143, 964233, 310906, 4676590]","[4801426, 4801426, 4801426, 2096639]",4676590,2096639,2609472,2096639,97.0,67,51,30


In [9]:
lt_df[0].describe()

,uid,target_iid,target_cid,next_iid,next_cid,knn_rank,NN_rank,DIN_rank,DIEN_rank
count,1.474300e+04,1.474300e+04,1.474300e+04,1.474300e+04,1.474300e+04,14743.000000,14743.000000,14743.000000,14743.000000
mean,5.028557e+05,2.577669e+06,2.686725e+06,2.573492e+06,2.698196e+06,30.146985,31.865902,47.880418,50.659296
std,2.939685e+05,1.469709e+06,1.396441e+06,1.479852e+06,1.392959e+06,28.070072,27.731505,28.048428,28.818221
min,7.000000e+00,1.260000e+03,9.874000e+03,7.640000e+02,5.064000e+03,1.000000,1.000000,1.000000,1.000000
25%,2.512495e+05,1.313104e+06,1.464116e+06,1.291732e+06,1.464116e+06,6.000000,8.000000,24.000000,26.000000
50%,4.985270e+05,2.579616e+06,2.465336e+06,2.566203e+06,2.520377e+06,20.000000,24.000000,46.000000,51.000000
75%,7.564575e+05,3.837116e+06,3.747017e+06,3.853076e+06,3.792551e+06,49.000000,51.000000,71.000000,76.000000
max,1.018006e+06,5.163006e+06,5.144469e+06,5.162806e+06,5.144469e+06,99.000000,99.000000,99.000000,99.000000
